In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------------- Model ---------------------------------- #
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GaussianNoise
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

import os
os.chdir("../")
from Sources.tools import *

# Data Loading

In [2]:
df = get_fundamental_df()

In [3]:
# Séparer les colonnes qui ne contiennent pas d'année
static_cols = ['symbol', 'company', 'sector', 'subsector']
data_cols = [col for col in df.columns if col not in static_cols]

# Filtrer les colonnes qui contiennent des années
filtered_cols = [col for col in data_cols if any(str(year) in col for year in range(2000, 2030))]

# Extraire les années et les variables
melted_df = df.melt(id_vars=static_cols, value_vars=filtered_cols, 
                     var_name="variable", value_name="value")

# Séparer le nom de la variable et l'année
melted_df[['variable_name', 'year']] = melted_df['variable'].str.rsplit('_', n=1, expand=True)

# Transformer l'année en format numérique
melted_df = melted_df.drop(columns=['variable']).pivot_table(
    index=['year'], 
    columns='variable_name', 
    values='value'
).reset_index()

In [4]:
melted_df

variable_name,year,asset_turnover,buyback_yield,capex_to_revenue,cash_ratio,cash_to_debt,cogs_to_revenue,current_ratio,days_inventory,debt_to_assets,...,price_to_earnings_ratio,price_to_earnings_ratio_nri,price_to_free_cashflow,price_to_operating_cashflow,rate_of_return,scaled_net_operating_assets,yoy_ebitda_growth,yoy_eps_growth,yoy_revenue_growth,zscore
0,2017,0.66,4.09,0.05,0.74,0.64,0.62,1.28,9.04,0.31,...,16.73,16.73,15.63,12.60,19.24,0.55,9.36,10.83,11.34,3.62
1,2018,0.72,6.71,0.05,0.57,0.58,0.62,1.13,9.82,0.31,...,18.94,18.94,17.60,14.58,17.24,0.41,19.40,29.40,21.69,4.22
2,2019,0.74,6.64,0.04,0.95,0.93,0.62,1.54,9.09,0.32,...,18.85,18.85,17.68,15.00,13.60,0.27,1.15,-0.34,5.36,4.24
3,2020,0.83,3.64,0.03,0.86,0.81,0.62,1.36,8.79,0.35,...,35.31,35.31,27.67,25.16,9.70,0.26,5.00,10.44,11.94,6.33
4,2021,1.08,3.65,0.03,0.50,0.50,0.58,1.07,9.12,0.36,...,25.22,25.22,25.67,22.94,16.13,0.39,57.96,71.04,38.50,6.99


In [15]:
technical_df = get_technical_df()

In [16]:
technical_df

,date,open,high,low,close,volume
7884,2017-01-03,4900.8501,4928.4902,4884.5200,4911.3301,1887670000
7885,2017-01-04,4920.7900,4944.7402,4919.7998,4937.2100,1885490000
7886,2017-01-05,4936.3501,4967.8999,4935.3398,4964.9502,1799170000
7887,2017-01-06,4973.8701,5020.7002,4957.8198,5007.0801,1711870000
7888,2017-01-09,5013.8198,5033.3198,5009.4502,5024.8999,1887740000
...,...,...,...,...,...,...
9389,2022-12-23,10911.0400,11007.0898,10830.9004,10985.4502,3544680000
9390,2022-12-27,10944.2998,10957.4404,10807.6104,10822.5098,3827290000
9391,2022-12-28,10807.1201,10898.6396,10671.1904,10679.3398,3842970000
9392,2022-12-29,10800.7998,10981.0596,10776.6201,10951.0498,4154100000


In [17]:
# Filtrer les années entre 2017 et 2022
technical_df = technical_df[(technical_df['date'].dt.year >= 2017) & (technical_df['date'].dt.year < 2022)]

# Sélectionner la première date de chaque année avec reset_index()
df_new_year = technical_df.loc[technical_df.groupby(technical_df['date'].dt.year)['date'].idxmin(), ['date', 'close']]

print(df_new_year)

           date       close
7884 2017-01-03   4911.3301
8135 2018-01-02   6511.3398
8386 2019-01-02   6360.8701
8638 2020-01-02   8872.2197
8891 2021-01-04  12694.6602


In [22]:
melted_df["close_price"] = df_new_year["close"].tolist()

In [28]:
melted_df.to_csv("fundamental_database.csv", sep=";", index=False)

# All symbols

In [2]:
df = get_fundamental_df()

In [3]:
# Liste des colonnes statiques (non liées aux années)
static_cols = ['symbol', 'company', 'sector', 'subsector']

# Obtenir les colonnes des années
data_cols = [col for col in df.columns if col not in static_cols]

# Créer une liste pour stocker les DataFrames filtrés par symbole
dfs_by_symbol = []

# Boucle sur chaque symbole unique
for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol].copy()  # Filtrer par symbole
    
    # Faire fondre (melt) les colonnes d'année pour chaque symbole
    melted_df = symbol_df.melt(id_vars=static_cols, value_vars=data_cols, 
                               var_name="variable", value_name="value")
    
    # Séparer le nom de la variable et l'année
    melted_df[['variable_name', 'year']] = melted_df['variable'].str.rsplit('_', n=1, expand=True)
    
    # Filtrer les lignes où l'année est valide (enlevons les "latest" et autres valeurs non numériques)
    melted_df = melted_df[melted_df['year'].notna()]  # Éliminer les NaN
    melted_df = melted_df[melted_df['year'].str.isnumeric()]  # Garder uniquement les lignes avec des années numériques
    
    # Convertir l'année en format numérique
    melted_df['year'] = melted_df['year'].astype(int)
    
    # Pivotement pour obtenir une structure de DataFrame avec les années comme index
    pivot_df = melted_df.drop(columns=['variable']).pivot_table(
        index=['year'], 
        columns='variable_name', 
        values='value'
    ).reset_index()

    # Interpolation linéaire des NaN dans chaque DataFrame de symbole
    pivot_df = pivot_df.interpolate(method='linear', axis=0)  # Interpolation linéaire sur les colonnes (variables)
    
    # Si des NaN persistent, on peut utiliser forward fill et backward fill
    pivot_df = pivot_df.ffill(axis=0).bfill(axis=0)  # Remplissage avant et arrière pour les bords
    
    # Ajouter le DataFrame traité à la liste
    dfs_by_symbol.append(pivot_df)

# Combiner tous les DataFrames dans une seule grande DataFrame
final_df = pd.concat(dfs_by_symbol)

# Trier les données par année
final_df = final_df.sort_values(by='year').reset_index(drop=True)

final_df = final_df.ffill(axis=0).bfill(axis=0)

In [4]:
final_df

variable_name,year,asset_turnover,buyback_yield,capex_to_revenue,cash_ratio,cash_to_debt,cogs_to_revenue,current_ratio,days_inventory,debt_to_assets,...,price_to_earnings_ratio,price_to_earnings_ratio_nri,price_to_free_cashflow,price_to_operating_cashflow,rate_of_return,scaled_net_operating_assets,yoy_ebitda_growth,yoy_eps_growth,yoy_revenue_growth,zscore
0,2017,0.66,4.09,0.05,0.74,0.64,0.62,1.28,9.04,0.31,...,16.73,16.73,15.63,12.60,19.24,0.55,9.36,10.83,11.34,3.62
1,2017,0.27,0.01,0.28,0.02,0.01,0.60,0.73,32.18,0.37,...,21.38,21.38,15.63,7.83,3.26,0.66,4.50,1.81,2.68,1.02
2,2017,1.71,6.26,0.01,0.18,0.26,0.75,1.07,36.59,0.20,...,21.56,21.56,14.89,12.12,11.59,0.52,-2.33,-1.05,1.91,4.06
3,2017,0.77,0.83,0.10,2.59,3.44,0.11,3.28,125.66,0.17,...,144.10,144.10,64.82,44.91,22.93,0.19,47.40,326.09,41.27,14.50
4,2017,0.44,5.56,0.04,1.55,1.00,0.17,1.57,125.66,0.82,...,31.10,31.10,21.75,20.22,18.22,-0.54,7.90,7.60,5.83,-5.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,2022,0.49,0.33,0.03,0.23,1.39,0.29,1.25,50.20,0.09,...,32.25,32.25,32.75,29.87,9.94,0.30,23.18,26.73,13.36,5.60
503,2022,0.46,5.04,0.01,0.82,0.51,0.27,1.56,50.20,0.58,...,32.25,32.25,185.51,156.46,9.94,0.32,-51.21,-45.95,18.53,1.27
504,2022,0.89,5.04,0.03,0.91,1.11,0.17,1.19,50.20,0.39,...,32.25,32.25,58.69,53.73,9.94,0.07,-4.59,13.26,32.25,8.87
505,2022,0.64,-1.81,0.04,3.43,51.26,0.26,3.91,50.20,0.01,...,34.28,34.28,32.32,29.39,9.94,0.09,57.36,100.00,50.75,17.76


In [5]:
technical_df = get_technical_df()

In [7]:
technical_df['date'] = pd.to_datetime(technical_df['date'])

# Filtrer les années entre 2017 et 2022
technical_df = technical_df[(technical_df['date'].dt.year >= 2017) & (technical_df['date'].dt.year <= 2022)]

# Sélectionner la première date de chaque année avec reset_index()
df_new_year = technical_df.loc[technical_df.groupby(technical_df['date'].dt.year)['date'].idxmin(), ['date', 'close']]

print(df_new_year)

           date       close
7884 2017-01-03   4911.3301
8135 2018-01-02   6511.3398
8386 2019-01-02   6360.8701
8638 2020-01-02   8872.2197
8891 2021-01-04  12694.6602
9143 2022-01-03  16501.7695


In [8]:
# Assurer que la colonne 'date' dans df_new_year est au format datetime
df_new_year['date'] = pd.to_datetime(df_new_year['date'])

# Extraire l'année de la colonne 'date' pour faciliter la fusion
df_new_year['year'] = df_new_year['date'].dt.year

# Maintenant, on va ajouter la colonne 'close' à final_df en fonction de l'année
# On va effectuer une jointure entre final_df et df_new_year sur la colonne 'year'

# Assurez-vous que final_df a bien une colonne 'year' avant cette étape.
final_df['year'] = final_df['year'].astype(int)  # S'assurer que 'year' est au format entier si nécessaire

# Fusionner les deux DataFrames sur l'année
final_df = final_df.merge(df_new_year[['year', 'close']], on='year', how='left')

In [9]:
final_df

,year,asset_turnover,buyback_yield,capex_to_revenue,cash_ratio,cash_to_debt,cogs_to_revenue,current_ratio,days_inventory,debt_to_assets,...,price_to_earnings_ratio_nri,price_to_free_cashflow,price_to_operating_cashflow,rate_of_return,scaled_net_operating_assets,yoy_ebitda_growth,yoy_eps_growth,yoy_revenue_growth,zscore,close
0,2017,0.66,4.09,0.05,0.74,0.64,0.62,1.28,9.04,0.31,...,16.73,15.63,12.60,19.24,0.55,9.36,10.83,11.34,3.62,4911.3301
1,2017,0.27,0.01,0.28,0.02,0.01,0.60,0.73,32.18,0.37,...,21.38,15.63,7.83,3.26,0.66,4.50,1.81,2.68,1.02,4911.3301
2,2017,1.71,6.26,0.01,0.18,0.26,0.75,1.07,36.59,0.20,...,21.56,14.89,12.12,11.59,0.52,-2.33,-1.05,1.91,4.06,4911.3301
3,2017,0.77,0.83,0.10,2.59,3.44,0.11,3.28,125.66,0.17,...,144.10,64.82,44.91,22.93,0.19,47.40,326.09,41.27,14.50,4911.3301
4,2017,0.44,5.56,0.04,1.55,1.00,0.17,1.57,125.66,0.82,...,31.10,21.75,20.22,18.22,-0.54,7.90,7.60,5.83,-5.25,4911.3301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,2022,0.49,0.33,0.03,0.23,1.39,0.29,1.25,50.20,0.09,...,32.25,32.75,29.87,9.94,0.30,23.18,26.73,13.36,5.60,16501.7695
503,2022,0.46,5.04,0.01,0.82,0.51,0.27,1.56,50.20,0.58,...,32.25,185.51,156.46,9.94,0.32,-51.21,-45.95,18.53,1.27,16501.7695
504,2022,0.89,5.04,0.03,0.91,1.11,0.17,1.19,50.20,0.39,...,32.25,58.69,53.73,9.94,0.07,-4.59,13.26,32.25,8.87,16501.7695
505,2022,0.64,-1.81,0.04,3.43,51.26,0.26,3.91,50.20,0.01,...,34.28,32.32,29.39,9.94,0.09,57.36,100.00,50.75,17.76,16501.7695


In [10]:
final_df.to_csv("fundamental_database_all_companies.csv", sep=";", index=False)